In [1]:
import mediapipe as mp
import cv2
import pyautogui
import time
import numpy as np
from datetime import datetime


mp_drawing = mp.solutions.drawing_utils #helps to render the landmarks
mp_hands = mp.solutions.hands

In [2]:
import sys
sys.executable

'c:\\users\\arham\\appdata\\local\\programs\\python\\python37\\python.exe'

In [3]:
cap = cv2.VideoCapture(0)
# You can setup your camera settings
cap.set(3,1920)
cap.set(4,1080)

True

In [4]:
'''while cap.isOpened():
    re, frame = cap.read()
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    print(image[:,:,0].shape)
    # cap.release()


cap.release()'''


'while cap.isOpened():\n    re, frame = cap.read()\n    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)\n    print(image[:,:,0].shape)\n    # cap.release()\n\n\ncap.release()'

In [5]:
def rotate_hue(val, deg):
    return (val + deg) % 180

In [12]:
def calibrate():
    cap = cv2.VideoCapture(0)
    cap.set(3,1920)
    cap.set(4,1080)
    
    start = datetime.now()
    end = datetime.now()
    
    cv2.namedWindow('image')
    #cv2.createTrackbar('a','image',0,255,nothing)
    
    #'Get Ready' screen
    ready_time = 7 #in seconds
    while (end - start).total_seconds() < ready_time:
            re, frame = cap.read()
            cv2.putText(frame, f'Keep hand still during calibration', (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            cv2.putText(frame, f'Place hand in view. Calibration begins in {ready_time-(end - start).total_seconds():.2f}', (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            end = datetime.now()
            cv2.imshow('image', frame)
            
            if cv2.waitKey(10) & 0xff == ord('q'):
                break
            
    angles = [i for i in range(360)]
    index = 0
    confidence_scores = []
    candidate_hues = []
    #run in static image mode so that palm detection fires every frame
    with mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.2 , min_tracking_confidence=0.5, max_num_hands=1) as hands:

        
        while cap.isOpened() and index < 360:

            re, frame = cap.read()

            # start the detection
            # ===================

            # convert the image to RGB
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

            hsv_frame = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            #angle = cv2.getTrackbarPos('a','image')
            angle = angles[index]
            hsv_frame[:, :, 0] = rotate_hue(hsv_frame[:, :, 0], angle)
            hsv_frame = np.array(hsv_frame, dtype=np.uint8)
            image = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

            # flip the image
            image = cv2.flip(image,1)

            image.flags.writeable = False

            # this is the main process
            results = hands.process(image)

            image.flags.writeable = True

            # print the results
            #print(results.multi_hand_landmarks)

            if results.multi_hand_landmarks:
                #print(results.multi_hand_landmarks)
                #break


                # lets change the colors and the dots and joits
                for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(255,0,0),thickness=2 , circle_radius=4),
                    mp_drawing.DrawingSpec(color=(0,255,255),thickness=2 , circle_radius=2))


                if results.palm_detections:
                    cv2.putText(image, f"Palm Detection Confidence: {results.palm_detections[0].score}", (440,80), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 10, 500))
                    confidence_scores.append(results.palm_detections[0].score[0])
                    candidate_hues.append(index)


            index += 1

            # recolor back the image to BGR
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
            cv2.imshow('image',image)

            if cv2.waitKey(10) & 0xff == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()
    
    if len(candidate_hues) > 0:
        print(len(confidence_scores))
        print(max(list(confidence_scores)))
        hue_select = np.amax(np.array(list(confidence_scores))).astype(int)
        print(hue_select)
        print(candidate_hues[hue_select])
        return candidate_hues[hue_select]
    else:
        print('Calibration failed.')
        return -1
    
    
calibrate()

16
0.8462724685668945
0
79


79

In [13]:
results = []

def nothing(x):
    pass

cap = cv2.VideoCapture(0)
# You can setup your camera settings
cap.set(3,1920)
cap.set(4,1080)

with mp_hands.Hands(min_detection_confidence=0.2 , min_tracking_confidence=0.5, max_num_hands=1) as hands:

    cv2.namedWindow('image')
    cv2.createTrackbar('a','image',0,360,nothing)
    while cap.isOpened():



        re, frame = cap.read()

        # start the detection
        # ===================

        # convert the image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        red  = image[:, :, 0]
        green  = image[:, :, 1]
        blue = image[:, :, 2]

        hsv_frame = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        angle = cv2.getTrackbarPos('a','image')
        hsv_frame[:, :, 0] = rotate_hue(hsv_frame[:, :, 0], angle)
        hsv_frame = np.array(hsv_frame, dtype=np.uint8)
        image = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

        # flip the image
        image = cv2.flip(image,1)

        image.flags.writeable = False

        # this is the main process
        results = hands.process(image)

        image.flags.writeable = True

        # print the results
        #print(results.multi_hand_landmarks)

        if results.multi_hand_landmarks:
            print(results.multi_hand_landmarks)
            #break

  
            # lets change the colors and the dots and joits
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image,hand,mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255,0,0),thickness=2 , circle_radius=4),
                mp_drawing.DrawingSpec(color=(0,255,255),thickness=2 , circle_radius=2))
            
            pointer_x = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
            pointer_y = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
            pointer_z = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z

            pointerMCP_x = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x
            pointerMCP_y = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y
            pointerMCP_z = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].z

            pinkyMCP_x = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].x
            pinkyMCP_y = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].y
            pinkyMCP_z = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].z


            cv2.putText(image, "pointer_tip", (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (300, 50, 50), 2)
            cv2.putText(image, "X: "+str(pointer_x)[:4], (40,80), cv2.FONT_HERSHEY_SIMPLEX, .8, (300, 50, 50))
            cv2.putText(image, "Y: "+str(pointer_y)[:4], (40,105), cv2.FONT_HERSHEY_SIMPLEX, .8, (300, 50, 50))
            cv2.putText(image, "Z: "+str(pointer_z)[:4], (40,130), cv2.FONT_HERSHEY_SIMPLEX, .8, (300, 50, 50))

            cv2.putText(image, "pointer_mcp", (220,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (30, 200, 50), 2)
            cv2.putText(image, "X: "+str(pointerMCP_x)[:4], (240,80), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 100, 50))
            cv2.putText(image, "Y: "+str(pointerMCP_y)[:4], (240,105), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 100, 50))
            cv2.putText(image, "Z: "+str(pointerMCP_z)[:7], (240,130), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 100, 50))

            cv2.putText(image, "pinky_mcp", (420,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (30, 20, 500), 2)
            cv2.putText(image, "X: "+str(pinkyMCP_x)[:4], (440,80), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 10, 500))
            cv2.putText(image, "Y: "+str(pinkyMCP_y)[:4], (440,105), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 10, 500))
            cv2.putText(image, "Z: "+str(pinkyMCP_z)[:7], (440,130), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 10, 500))
            
            if results.palm_detections:
                cv2.putText(image, f"Palm Detection Confidence: {results.palm_detections[0].score}", (440,80), cv2.FONT_HERSHEY_SIMPLEX, .8, (30, 10, 500))
            

            '''move = "center"
            if pointer_x > .7:
                move = "right"
                pyautogui.moveRel(-10-500*(.3 - pointer_y), 0)
            if pointer_x < .3:
                move = "left"
                pyautogui.moveRel(10+500*(pointer_y-.7), 0)
            if pointer_y > .5:
                move = "down"
                pyautogui.moveRel(0, 5+100*(pointer_y-.5))
            if pointer_y < .3:
                move = "up"
                pyautogui.moveRel(0, -5-100*(.3 - pointer_y))

            cv2.putText(image, move, (50,160), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

            
            if pointer_y > pointerMCP_y:
                # pyautogui.click()
                pyautogui.mouseDown()
                cv2.putText(image, "clicked", (50,175), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            else:
                pyautogui.mouseUp()'''
            
            # if pinkyMCP_z < pointerMCP_z - .0:
            #     cv2.destroyAllWindows()
            #     break





        # recolor back the image to BGR
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        cv2.imshow('image',image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            print(results)
            break
        
cap.release()
cv2.destroyAllWindows()

[landmark {
  x: 0.2575802803039551
  y: 0.7533608078956604
  z: 0.0
}
landmark {
  x: 0.22774283587932587
  y: 0.7083171606063843
  z: -0.0018929868238046765
}
landmark {
  x: 0.20768797397613525
  y: 0.6568312048912048
  z: -0.01375382300466299
}
landmark {
  x: 0.1828635334968567
  y: 0.6375685930252075
  z: -0.027821825817227364
}
landmark {
  x: 0.1588491052389145
  y: 0.6348291039466858
  z: -0.04326373711228371
}
landmark {
  x: 0.2580074369907379
  y: 0.57337486743927
  z: -0.017959872260689735
}
landmark {
  x: 0.2719932794570923
  y: 0.47988152503967285
  z: -0.031591128557920456
}
landmark {
  x: 0.28123125433921814
  y: 0.4269380271434784
  z: -0.03922430798411369
}
landmark {
  x: 0.2866939902305603
  y: 0.39185553789138794
  z: -0.043826255947351456
}
landmark {
  x: 0.2860569953918457
  y: 0.5866879224777222
  z: -0.028003893792629242
}
landmark {
  x: 0.3158942759037018
  y: 0.49133893847465515
  z: -0.039891090244054794
}
landmark {
  x: 0.3362758457660675
  y: 0.43690

[landmark {
  x: 0.2702004313468933
  y: 0.7301759123802185
  z: 0.0
}
landmark {
  x: 0.2209581732749939
  y: 0.6960175037384033
  z: -0.014782002195715904
}
landmark {
  x: 0.18351992964744568
  y: 0.6436889171600342
  z: -0.030104560777544975
}
landmark {
  x: 0.1662663221359253
  y: 0.6268611550331116
  z: -0.04565080255270004
}
landmark {
  x: 0.16242578625679016
  y: 0.6415363550186157
  z: -0.06301229447126389
}
landmark {
  x: 0.2173272669315338
  y: 0.5322661399841309
  z: -0.033773962408304214
}
landmark {
  x: 0.208809494972229
  y: 0.4325200915336609
  z: -0.05492763966321945
}
landmark {
  x: 0.20513103902339935
  y: 0.3661099374294281
  z: -0.06800688803195953
}
landmark {
  x: 0.2031419277191162
  y: 0.3149004578590393
  z: -0.0765901431441307
}
landmark {
  x: 0.2568579912185669
  y: 0.5230095386505127
  z: -0.04153657704591751
}
landmark {
  x: 0.2562062442302704
  y: 0.4105539619922638
  z: -0.06239274889230728
}
landmark {
  x: 0.25604474544525146
  y: 0.335816770792

[landmark {
  x: 0.28763672709465027
  y: 0.7336227893829346
  z: 0.0
}
landmark {
  x: 0.25063708424568176
  y: 0.6743043065071106
  z: -0.00603938614949584
}
landmark {
  x: 0.22850701212882996
  y: 0.5861923098564148
  z: -0.013286540284752846
}
landmark {
  x: 0.23367612063884735
  y: 0.5155301690101624
  z: -0.02218383550643921
}
landmark {
  x: 0.24451392889022827
  y: 0.4726181626319885
  z: -0.0318925678730011
}
landmark {
  x: 0.22629661858081818
  y: 0.5239923596382141
  z: -0.022742703557014465
}
landmark {
  x: 0.20397359132766724
  y: 0.4123624861240387
  z: -0.04191303253173828
}
landmark {
  x: 0.19437450170516968
  y: 0.3332520127296448
  z: -0.05316942557692528
}
landmark {
  x: 0.18854796886444092
  y: 0.26824769377708435
  z: -0.06012948602437973
}
landmark {
  x: 0.2583003342151642
  y: 0.5096282362937927
  z: -0.032808225601911545
}
landmark {
  x: 0.25056132674217224
  y: 0.38981080055236816
  z: -0.051945336163043976
}
landmark {
  x: 0.2484341561794281
  y: 0.30

[landmark {
  x: 0.33439069986343384
  y: 0.8151755928993225
  z: 0.0
}
landmark {
  x: 0.2934825122356415
  y: 0.7581376433372498
  z: -0.010860306210815907
}
landmark {
  x: 0.2645452916622162
  y: 0.6695681214332581
  z: -0.026019740849733353
}
landmark {
  x: 0.2592869997024536
  y: 0.6026841402053833
  z: -0.04439876228570938
}
landmark {
  x: 0.25615793466567993
  y: 0.5571627020835876
  z: -0.06362085789442062
}
landmark {
  x: 0.25651752948760986
  y: 0.5717366933822632
  z: -0.03755578398704529
}
landmark {
  x: 0.240077406167984
  y: 0.4327348470687866
  z: -0.06718436628580093
}
landmark {
  x: 0.23542790114879608
  y: 0.3455027937889099
  z: -0.0866210013628006
}
landmark {
  x: 0.23334480822086334
  y: 0.2761787176132202
  z: -0.09973052889108658
}
landmark {
  x: 0.28913676738739014
  y: 0.5702667236328125
  z: -0.053337518125772476
}
landmark {
  x: 0.2942099869251251
  y: 0.42677608132362366
  z: -0.0852343738079071
}
landmark {
  x: 0.3009026348590851
  y: 0.3249705433

[landmark {
  x: 0.5501415133476257
  y: 0.7748081088066101
  z: 0.0
}
landmark {
  x: 0.5047072172164917
  y: 0.7462330460548401
  z: -0.021898377686738968
}
landmark {
  x: 0.46595653891563416
  y: 0.7105922698974609
  z: -0.04844832792878151
}
landmark {
  x: 0.43087446689605713
  y: 0.7170348167419434
  z: -0.07737625390291214
}
landmark {
  x: 0.39401036500930786
  y: 0.7465286254882812
  z: -0.10824665427207947
}
landmark {
  x: 0.4726102352142334
  y: 0.540336012840271
  z: -0.03904273733496666
}
landmark {
  x: 0.4676094055175781
  y: 0.4100602865219116
  z: -0.06938993185758591
}
landmark {
  x: 0.4643057882785797
  y: 0.33133336901664734
  z: -0.09261906892061234
}
landmark {
  x: 0.46241772174835205
  y: 0.26903265714645386
  z: -0.10886397957801819
}
landmark {
  x: 0.5106182098388672
  y: 0.5325137376785278
  z: -0.0507669635117054
}
landmark {
  x: 0.5185632705688477
  y: 0.4002629518508911
  z: -0.07980892062187195
}
landmark {
  x: 0.5273433327674866
  y: 0.321275889873

...

[landmark {
  x: 0.26471617817878723
  y: 0.6959637403488159
  z: 0.0
}
landmark {
  x: 0.31609871983528137
  y: 0.6731014251708984
  z: 0.012029780074954033
}
landmark {
  x: 0.34865739941596985
  y: 0.6242042183876038
  z: 0.010673106648027897
}
landmark {
  x: 0.3766425549983978
  y: 0.5747259855270386
  z: 0.0074234772473573685
}
landmark {
  x: 0.4037606716156006
  y: 0.5506525635719299
  z: 0.0032387503888458014
}
landmark {
  x: 0.35078105330467224
  y: 0.4871152639389038
  z: -0.014769468456506729
}
landmark {
  x: 0.38023704290390015
  y: 0.3735203146934509
  z: -0.02293754369020462
}
landmark {
  x: 0.39458411931991577
  y: 0.3038606345653534
  z: -0.025593120604753494
}
landmark {
  x: 0.40614408254623413
  y: 0.24633480608463287
  z: -0.02840162254869938
}
landmark {
  x: 0.3177053928375244
  y: 0.46031004190444946
  z: -0.028093568980693817
}
landmark {
  x: 0.33733421564102173
  y: 0.33405157923698425
  z: -0.03977564349770546
}
landmark {
  x: 0.34443914890289307
  y: 0.

[landmark {
  x: 0.20337074995040894
  y: 0.7098166346549988
  z: 0.0
}
landmark {
  x: 0.172540083527565
  y: 0.6577440500259399
  z: 0.016924923285841942
}
landmark {
  x: 0.16897420585155487
  y: 0.56769859790802
  z: 0.023760268464684486
}
landmark {
  x: 0.17544108629226685
  y: 0.5044712424278259
  z: 0.021585239097476006
}
landmark {
  x: 0.17244137823581696
  y: 0.450595498085022
  z: 0.01858377642929554
}
landmark {
  x: 0.2125362902879715
  y: 0.43868234753608704
  z: 0.03982728347182274
}
landmark {
  x: 0.22717127203941345
  y: 0.32817500829696655
  z: 0.033143818378448486
}
landmark {
  x: 0.23682014644145966
  y: 0.26251110434532166
  z: 0.026372043415904045
}
landmark {
  x: 0.24446932971477509
  y: 0.20979733765125275
  z: 0.0219219122081995
}
landmark {
  x: 0.23129644989967346
  y: 0.4273560643196106
  z: 0.018181363120675087
}
landmark {
  x: 0.25368860363960266
  y: 0.30490052700042725
  z: 0.01303877867758274
}
landmark {
  x: 0.2713377773761749
  y: 0.231184482574

[landmark {
  x: 0.3226621150970459
  y: 0.7306811213493347
  z: 0.0
}
landmark {
  x: 0.2697823941707611
  y: 0.7153263092041016
  z: -0.030878696590662003
}
landmark {
  x: 0.22040188312530518
  y: 0.6686005592346191
  z: -0.05779807269573212
}
landmark {
  x: 0.17816177010536194
  y: 0.6598386168479919
  z: -0.08342950791120529
}
landmark {
  x: 0.13897384703159332
  y: 0.6699689626693726
  z: -0.11034402996301651
}
landmark {
  x: 0.24982112646102905
  y: 0.48200899362564087
  z: -0.04670356214046478
}
landmark {
  x: 0.2299489974975586
  y: 0.3462304472923279
  z: -0.07619402557611465
}
landmark {
  x: 0.21852612495422363
  y: 0.25627490878105164
  z: -0.09813453257083893
}
landmark {
  x: 0.21030092239379883
  y: 0.1831979751586914
  z: -0.11309004575014114
}
landmark {
  x: 0.288895845413208
  y: 0.4594636857509613
  z: -0.05090208724141121
}
landmark {
  x: 0.2784920930862427
  y: 0.31316831707954407
  z: -0.0776880532503128
}
landmark {
  x: 0.27217358350753784
  y: 0.21390229

[landmark {
  x: 0.32304847240448
  y: 0.7100868821144104
  z: 0.0
}
landmark {
  x: 0.27139437198638916
  y: 0.7039370536804199
  z: -0.019380444660782814
}
landmark {
  x: 0.22691991925239563
  y: 0.6585631370544434
  z: -0.04143745079636574
}
landmark {
  x: 0.19153785705566406
  y: 0.6431871652603149
  z: -0.06374180316925049
}
landmark {
  x: 0.15542353689670563
  y: 0.6459567546844482
  z: -0.08744850754737854
}
landmark {
  x: 0.24607577919960022
  y: 0.48600342869758606
  z: -0.039365217089653015
}
landmark {
  x: 0.22495058178901672
  y: 0.35308486223220825
  z: -0.06554774194955826
}
landmark {
  x: 0.2148839384317398
  y: 0.26330631971359253
  z: -0.08423805981874466
}
landmark {
  x: 0.2078096568584442
  y: 0.19350889325141907
  z: -0.0968523696064949
}
landmark {
  x: 0.2840381860733032
  y: 0.4615846872329712
  z: -0.047356124967336655
}
landmark {
  x: 0.2732837200164795
  y: 0.3157384395599365
  z: -0.07186106592416763
}
landmark {
  x: 0.26931247115135193
  y: 0.217339

<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [ ]:
results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]


In [10]:
cap.release()
cv2.destroyAllWindows()